# Regularisation

モデルパラメータへの正則化の適用は簡単だ。  
ちょうど`norm`のような適切な正則化器を各モデルパラメータに適用し、結果を全体損失に追加するだけだ。

例えば、簡単な回帰があるとする。

In [42]:
using Flux
using Flux: crossentropy
using LinearAlgebra

m = Dense(10,5)
loss(x,y) = crossentropy(softmax(m(x)), y)

loss (generic function with 1 method)

パラメータ（`m.W`と`m.b`）のL2ノルムを取って正則化ができる。

In [45]:
penalty() = norm(m.W) + norm(m.b)
loss(x,y) = crossentropy(softmax(m(x)), y) + penalty()

loss (generic function with 1 method)

レイヤを扱うとき、Fluxはすべてのパラメータを一度に掴むための`params`関数を提供する。  
`sum(norm, params)`で簡単にペナルティを課せる。

In [16]:
params(m)

2-element Array{Any,1}:
 Flux.Tracker.TrackedReal{Float64}[-0.376348 (tracked) -0.00425538 (tracked) … -0.371803 (tracked) 0.0844787 (tracked); 0.389778 (tracked) 0.219423 (tracked) … -0.491801 (tracked) -0.519534 (tracked); … ; 0.540866 (tracked) -0.609542 (tracked) … -0.405648 (tracked) -0.493111 (tracked); 0.502622 (tracked) -0.623666 (tracked) … -0.318689 (tracked) -0.400307 (tracked)]
 Flux.Tracker.TrackedReal{Float64}[0.0 (tracked), 0.0 (tracked), 0.0 (tracked), 0.0 (tracked), 0.0 (tracked)]                                                                                                                                                                                                                                                                   

In [18]:
sum(norm, params(m))

2.771588008595627 (tracked)

これは多層パーセプトロンを使ったより大きな例。

In [52]:
m = Chain(
    Dense(28^2, 128, relu),
    Dense(128, 32, relu),
    Dense(32, 10), # もとのコードだとこの行にsoftmaxも並んでいるがカッコの位置を見落として苦しんだので注意
    softmax)

loss(x,y) = crossentropy(m(x), y) + sum(norm, params(m))

loss(rand(28^2), rand(10))

35.13749596450304 (tracked)

`activations`関数を使い、レイヤごとに正則化を追加することも簡単にできる。

In [53]:
c = Chain(Dense(10,5,σ), Dense(5,2), softmax)

Chain(Dense(10, 5, NNlib.σ), Dense(5, 2), NNlib.softmax)

In [63]:
using Flux: activations
activations(c, rand(10))

3-element Array{Any,1}:
 Flux.Tracker.TrackedReal{Float64}[0.576815 (tracked), 0.177552 (tracked), 0.717878 (tracked), 0.556235 (tracked), 0.318195 (tracked)]
 Flux.Tracker.TrackedReal{Float64}[0.924826 (tracked), 0.384222 (tracked)]                                                            
 Flux.Tracker.TrackedReal{Float64}[0.631953 (tracked), 0.368047 (tracked)]                                                            

In [64]:
sum(norm, ans) # ansは直前に実行した関数の出力が入っている

2.868664819289895 (tracked)